In [5]:
import torch
import huggingface
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
from trl import PPOTrainer, PPOConfig
import sys
import os
sys.path.append(os.path.abspath(".."))
from json_reader import JSONFilter

In [10]:
reward_model = AutoModelForSequenceClassification.from_pretrained("gpt2", num_labels=1)
reward_model.load_state_dict(torch.load("../models/generator_reward_model.pth", map_location=torch.device('cpu')))

Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


<All keys matched successfully>

In [8]:
reward_model.eval()
for param in reward_model.parameters():
    param.requires_grad = False

In [37]:
tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen2.5-Coder-0.5B")

In [36]:
from agraph import create_workflow

policy_model = create_workflow()

Sliding Window Attention is enabled but not implemented for `sdpa`; unexpected results may be encountered.
Some parameters are on the meta device because they were offloaded to the disk.
Some parameters are on the meta device because they were offloaded to the disk.
Some parameters are on the meta device because they were offloaded to the disk.


In [ ]:
class LangGraphPolicyWrapper(torch.nn.Module):
    def __init__(self, policy_model):
        super().__init__()
        self.policy_model = policy_model
        
    def forward(self, prompt):
        output = self.policy_model.forward(prompt)
        return output

policy_model = LangGraphPolicyWrapper(policy_model)

In [53]:
train_dataset = {}

In [54]:

model_name = policy_model 

config = PPOConfig(
    learning_rate=1e-5,
    batch_size=4,        
    mini_batch_size=2,     
    gradient_accumulation_steps=1,
    num_ppo_epochs=4,       
    gamma=1.0,
    lam=0.95,             
    cliprange=0.2,
    cliprange_value=0.2,
    vf_coef=0.1,
    kl_coef=0.05,          
    seed=42
)

ppo_trainer = PPOTrainer(
    args=config,                 
    processing_class=tokenizer,   
    model=policy_model,           
    ref_model=None,                
    reward_model=reward_model,     
    train_dataset=train_dataset    
)

AttributeError: 'LangGraphPolicyWrapper' object has no attribute 'generation_config'